# Dub any and all videos!

* Github: https://github.com/thehomeworkclub/DeepDubbed
* Demo: Here! (Scroll Down)
* Wav2Lip: https://arxiv.org/abs/2008.10010





In [ ]:
#@title <h1>Step1: Setup Wav2Lip</h1>
# @markdown ## Please click the run button to start the project. This will:
# @markdown
# @markdown * Install dependencies
# @markdown *  Prepare the model
from IPython.display import HTML, clear_output
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/justinjohn0306/Wav2Lip

%cd /content/Wav2Lip

#download the pretrained model
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip.pth' -O 'checkpoints/wav2lip.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/resnet50.pth' -O 'checkpoints/resnet50.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth' -O 'checkpoints/mobilenet.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl
!pip install git+https://github.com/elliottzheng/batch-face.git@master

!pip install ffmpeg-python mediapipe==0.8.11

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

%cd /
from ghc.l_ghc_cf import l_ghc_cf
%cd content

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr


from IPython.display import HTML
from base64 import b64encode
def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

from IPython.display import clear_output

clear_output()
print("All set and ready!")

# Dub Youtube Video

In [ ]:
#@title STEP2: Select a Youtube Video

# Install yt-dlp and ffmpeg
import os
!pip install yt-dlp
!apt install ffmpeg

# Create a new folder 'video_data' if it does not exist
if not os.path.exists('/content/video_data'):
    os.makedirs('/content/video_data')

#@markdown ## Find YouTube video ID from URL
#@markdown ___
#@markdown Link format:
#@markdown ``https://youtu.be/vAnWYLTdvfY`` ❌
#@markdown ``https://www.youtube.com/watch?v=vAnWYLTdvfY`` ✔️

!rm -df youtube.mp4

#@markdown ___
from urllib import parse as urlparse
YOUTUBE_URL = 'https://www.youtube.com/watch?v=vAnWYLTdvfY' #@param {type:"string"}
url_data = urlparse.urlparse(YOUTUBE_URL)
query = urlparse.parse_qs(url_data.query)
YOUTUBE_ID = query["v"][0]

# remove previous input video and audio from 'video_data'
!rm -f /content/video_data/input_vid.mp4
!rm -f /content/video_data/input_audio.wav

#@markdown ___
#@markdown ### Trim the video (start, end) seconds
start = 35 #@param {type:"integer"}
end = 62 #@param {type:"integer"}
interval = end - start

#@markdown <font color="orange"> Note: ``the trimmed video must have face on all frames``

# Download the YouTube video using yt-dlp with both video and audio
!yt-dlp -f 'best' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

# Cut the video using FFmpeg
!ffmpeg -y -i youtube.mp4 -ss {start} -t {interval} -async 1 /content/video_data/input_vid.mp4

# Check if the video has an audio stream
video_info = !ffprobe -v error -select_streams a -show_entries stream=codec_type -of default=noprint_wrappers=1:nokey=1 /content/video_data/input_vid.mp4
if 'audio' in video_info:
    # Extract audio from the trimmed video and save as WAV
    !ffmpeg -i /content/video_data/input_vid.mp4 -ab 160k -ac 2 -ar 44100 -vn /content/video_data/input_audio.wav
else:
    print("No audio stream found in the video.")

# Preview the trimmed video
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/video_data/input_vid.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""<video width=600 controls><source src="{data_url}"></video>""")


In [ ]:
#@title STEP3: Clone Voice using Eleven Labs API

# Install necessary packages
!pip install yt-dlp elevenlabs googletrans==4.0.0rc1 requests

import os
from elevenlabs import clone, generate, play, set_api_key
from urllib import parse as urlparse
from googletrans import Translator

set_api_key("842b9d0f5b3570dae2fdb7e2844763e1")
api_key = "842b9d0f5b3570dae2fdb7e2844763e1"
translator = Translator()
#@markdown ### Required Inputs for Voice Cloning
voice_name = 'Donald Trump' #@param {type:"string"}
voice_description = 'Distinct Queens New York accent, confident and assertive tone, rhythmic speech pattern with key word emphasis, persuasive and bold language use, moderate to deep pitch, somewhat raspy quality, employs pauses for emphasis, varies pitch for emotional conveyance, mix of formality and off-the-cuff remarks, often uses repetition for emphasis. Speaking very enthusiastically and fiercely' #@param {type:"string"}
#@markdown ### Enter the Script for Voice Synthesis
script_to_synthesize = "My name is donald trump and I approve of this hackathon submission." #@param {type:"string"}
translation_lang = 'en' #@param ['en', 'zh-cn', 'es', 'hi', 'pt', 'fr', 'de', 'ja', 'ar', 'ko', 'id', 'it', 'nl', 'tr', 'pl', 'sv', 'tl', 'ms', 'ru', 'ro', 'uk', 'el', 'cs', 'da', 'fi', 'bg', 'hr', 'sk', 'ta']
if translation_lang != 'en':
  try:
    trans = translator.translate(script_to_synthesize, dest=translation_lang).text
    script_to_synthesize = trans
  except Exception as e:
    print(e)
    print("Unable to translate to the destination language! Continuing with English. You can also go to googletranslate and paste its output here")
#@markdown ### Optional YouTube Links for Additional Voice Data (Leave blank if not used)
youtube_link_1 = 'https://www.youtube.com/watch?v=hDNiNdsPHNA' #@param {type:"string"}
youtube_link_2 = 'https://www.youtube.com/watch?v=NMoL-fZK64A' #@param {type:"string"}
youtube_link_3 = 'https://www.youtube.com/watch?v=Am1DfjmGcvc' #@param {type:"string"}
youtube_link_4 = 'https://www.youtube.com/watch?v=N0CSUhF4tyA' #@param {type:"string"}

# Function to download YouTube video audio directly in MP3 format
def download_youtube_audio(youtube_link, output_file):
    if youtube_link:
        # Extract YouTube ID from the link
        url_data = urlparse.urlparse(youtube_link)
        query = urlparse.parse_qs(url_data.query)
        youtube_id = query.get("v")
        if youtube_id:
            # Correctly format the URL using f-string
            command = f"yt-dlp -x --audio-format mp3 --output '{output_file}.%(ext)s' 'https://www.youtube.com/watch?v={youtube_id[0]}'"
            os.system(command)

# Directory to store additional audio files
additional_audio_dir = '/content/video_data/voice_data'
if not os.path.exists(additional_audio_dir):
    os.makedirs(additional_audio_dir)

# Process optional YouTube links and extract audio
additional_audio_files = ["/content/video_data/input_audio.wav"]
for i, link in enumerate([youtube_link_1, youtube_link_2, youtube_link_3, youtube_link_4], start=1):
    if link.strip():
        output_file = f'{additional_audio_dir}/youtube_audio_{i}'
        download_youtube_audio(link, output_file)
        if os.path.exists(output_file + '.mp3'):
            additional_audio_files.append(output_file + '.mp3')

# Check if we have at least one audio file
if not additional_audio_files:
    raise Exception("No audio files were downloaded. Please check the YouTube links.")

# Initialize Eleven Labs API client
# Note: Replace 'your_api_key_here' with your actual API key
voice = clone(
    name=voice_name,
    description=voice_description,
    files=additional_audio_files  # List of audio files for voice cloning
)

# Generate audio using the cloned voice
audio = generate(text=script_to_synthesize, voice=voice, model='eleven_multilingual_v2')

# Save the generated audio
output_audio_path = '/content/video_data/output_audio.mp3'
with open(output_audio_path, "wb") as file:
    file.write(audio)

print(f"Audio file saved in '{output_audio_path}'")


In [ ]:
#@title STEP4: Start Crunching and Preview Output
#@markdown <b>Note: Only change these if you have to</b>

%cd /content/Wav2Lip

# Set up paths and variables for the output file
output_file_path = '/content/Wav2Lip/results/result_voice.mp4'
import requests
api_key = "842b9d0f5b3570dae2fdb7e2844763e1"
# Delete existing output file before processing, if any
if os.path.exists(output_file_path):
    os.remove(output_file_path)

pad_top =  0#@param {type:"integer"}
pad_bottom =  10#@param {type:"integer"}
pad_left =  0#@param {type:"integer"}
pad_right =  0#@param {type:"integer"}
rescaleFactor =  1#@param {type:"integer"}
nosmooth = True #@param {type:"boolean"}
#@markdown ___
#@markdown Model selection:
use_hd_model = False #@param {type:"boolean"}
checkpoint_path = 'checkpoints/wav2lip.pth' if not use_hd_model else 'checkpoints/wav2lip_gan.pth'

# Update paths for the input video and audio
input_video_path = '/content/video_data/input_vid.mp4'
input_audio_path = '/content/video_data/output_audio.mp3'

if nosmooth == False:
  !python inference.py --checkpoint_path $checkpoint_path --face "$input_video_path" --audio "$input_audio_path" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor
else:
  !python inference.py --checkpoint_path $checkpoint_path --face "$input_video_path" --audio "$input_audio_path" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor --nosmooth

# Preview output video
if os.path.exists(output_file_path):
    from IPython.display import HTML, clear_output
    from base64 import b64encode
    clear_output()
    print("Final Video Preview")
    print("Download this video from", output_file_path)
    mp4 = open(output_file_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML(f"""<video width=600 controls><source src="{data_url}" type="video/mp4"></video>"""))
    response = requests.delete(
      f"https://api.elevenlabs.io/v1/voices/{voice.voice_id}",
      headers={
          "accept": "application/json",
          "xi-api-key": api_key
      }
    )
    if response.status_code == 200:
      print("Voice deleted successfully.")
    else:
      print("Failed to delete voice.")
else:
    print("Processing failed. Output video not found.")
